# **DDPM — 학습/추론/시각화**

**DDPM (Denoising Diffusion Probabilistic Models)** 을 직접 구현하면서 구조를 이해하는 것을 목표로 합니다.

## 학습 목표
- DDPM의 원리를 이해한다.
- Forward / Reverse diffusion 을 이해하고 Training과 Sampling 과정을 구현한다.

---

### 구성 개요
1. Diffusion / DDPM 개념 요약
2. 실험 환경 및 기본 설정
3. U-Net 기반 노이즈 예측 신경망 구현
4. Forward Diffusion Process 정의 ($q$)
5. Reverse Diffusion Process 정의 ($p$)
6. Fashion-MNIST 데이터셋 로드 및 전처리
7. 학습 루프 구현
8. 샘플링 및 시각화 (이미지 + GIF)


## 1. Diffusion / DDPM 개념 요약

### 1.1 생성 모델 관점

- 생성 모델(generative model)은 **데이터 분포 $p_{data}(x)$** 를 학습해서, 학습 후에는 샘플링으로 새로운 데이터를 생성할 수 있는 모델입니다.
- 예: GAN, VAE, Autoregressive model, Flow-based model, Diffusion model 등

### 1.2 Diffusion 모델의 아이디어 (고수준)

1. **Forward Diffusion (noising)** - 깨끗한 이미지 $x_0$ 에 **점진적으로 가우시안 노이즈를 추가**해서 $x_1, x_2, \dots, x_T$ 를 만듭니다.
   - $T$ 가 충분히 크면, $x_T$ 는 거의 **표준 정규분포와 비슷한 pure noise** 가 됩니다.
   - 이 과정은 **고정된 마르코프 체인** $q(x_t \mid x_{t-1})$ 으로 정의됩니다.

2. **Reverse Diffusion (denoising)** - 우리가 학습하고 싶은 것은 반대로 **노이즈에서 시작해서 점점 이미지를 되살리는 과정**입니다.
   - $p_\theta(x_{t-1} \mid x_t)$ 를 신경망으로 근사해서, $x_T \sim \mathcal{N}(0, I)$ 에서 시작해 $x_0$ 를 샘플링할 수 있게 합니다.

3. **DDPM의 핵심 포인트** - Forward process는 **닫힌형태(analytic form)** 를 가지도록 설계되어, $q(x_t \mid x_0)$ 를 한 번에 샘플링할 수 있습니다.
   - 학습 시에는 $x_0$ 와 $t$ 를 샘플링한 뒤, $x_t$ 를 계산하고, 네트워크가 **노이즈 $\epsilon$** 을 예측하도록 만듭니다.
   - 손실 함수는 보통 **예측 노이즈와 실제 노이즈의 $L_2$ 또는 Huber loss** 로 정의합니다.

이제부터는 이 개념을 실제 코드로 구현해 봅니다.

## 2. 환경 설정 및 기본 Import

- PyTorch, torchvision, einops, datasets, matplotlib 등을 사용합니다.
- 아래 셀은 GPU 사용 가능 여부를 확인하고, 필요한 라이브러리를 import 합니다.


In [ ]:
# 유틸리티 및 함수형 프로그래밍 관련
import math
from functools import partial
from pathlib import Path   # 파일 및 디렉토리 경로 관리

# PyTorch 핵심 라이브러리
import torch
import torch.nn as nn            # 신경망 레이어 구성 (Conv2d, Linear 등)
import torch.nn.functional as F   # 활성화 함수 및 손실 함수

# 데이터 로딩 및 이미지 처리
from torch.utils.data import DataLoader
from torchvision import transforms      # 이미지 전처리 (Resize, ToTensor 등)
from torchvision.utils import save_image # 결과 이미지 저장

# 텐서 차원 조작 (가독성 높은 텐서 변환)
from einops import rearrange, reduce
from einops.layers.torch import Rearrange # 모델 레이어 내 차원 재구성

# 데이터셋 불러오기 및 시각화/수치 계산
from datasets import load_dataset # 데이터셋 로드
import numpy as np
import matplotlib.pyplot as plt   # 학습 곡선 및 이미지 시각화
from tqdm.auto import tqdm        # 학습 진행률 표시바

# 연산 장치 설정 (GPU 가속 여부 확인)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


## 3. U-Net 구현을 위한 Helper 함수들

DDPM에서 사용할 U-Net 구조를 구현하기 위해 아래와 같은 helper 함수/클래스를 사용합니다.

- `exists`, `default`: 인자 존재 여부/기본값 처리용 유틸리티
- `num_to_groups`: 전체 개수를 mini-batch 그룹 형태로 나누기
- `Residual`: 잔차 연결(residual connection)을 감싸는 래퍼
- `Upsample` / `Downsample`: 이미지 해상도를 2배 업/다운샘플링하는 모듈


In [ ]:
def exists(x):
    """값이 존재하는지(None 여부) 확인하는 유틸리티"""
    return x is not None

def default(val, d):
    """val이 None일 경우 기본값 d를 반환 (d가 함수라면 호출해서 결과 반환)"""
    if exists(val):
        return val
    return d() if callable(d) else d

def num_to_groups(num, divisor):
    """
    전체 샘플(num)을 지정된 크기(divisor)로 나눈 리스트를 생성.
    예: num=10, divisor=4 -> [4, 4, 2] (Batch를 나누어 처리할 때 유용)
    """
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

class Residual(nn.Module):
    """
    입력값 x를 출력에 다시 더해주는 잔차 연결(Skip Connection) 래퍼.
    Deep Network에서 Gradient Vanishing 문제를 완화하고 학습 안정성을 높임.
    """
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, *args, **kwargs):
        # f(x) + x: 입력과 출력의 Shape이 동일해야 함
        return self.fn(x, *args, **kwargs) + x

def Upsample(dim, dim_out=None):
    """
    이미지 해상도를 2배 키우는 업샘플링 블록.
    Nearest Neighbor로 해상도를 올린 후 Conv2d로 특징을 부드럽게 정제(Anti-aliasing).
    """
    return nn.Sequential(
        nn.Upsample(scale_factor=2, mode="nearest"),
        nn.Conv2d(dim, default(dim_out, dim), 3, padding=1),
    )

def Downsample(dim, dim_out=None):
    """
    이미지 해상도를 1/2로 줄이는 다운샘플링 블록.
    Space-to-Depth(Pixel Unshuffle) 방식을 사용하여 정보 손실 없이 채널을 확장한 후,
    1x1 Conv를 통해 원하는 채널 크기(dim_out)로 압축함.
    """
    return nn.Sequential(
        # (B, C, H*2, W*2) -> (B, C*4, H, W)
        Rearrange("b c (h p1) (w p2) -> b (c p1 p2) h w", p1=2, p2=2),
        # 1x1 Conv로 채널 수 조정
        nn.Conv2d(dim * 4, default(dim_out, dim), 1),
    )

## 4. Time Embedding과 ResNet Block

### 4.1 Sinusoidal Position Embedding

- Diffusion model은 **time step(노이즈 단계) $t$** 에 따라 같은 네트워크 파라미터를 공유합니다.
- 네트워크가 “지금 몇 번째 노이즈 단계인지” 알 수 있도록, Transformer에서 사용하는 **sinusoidal position embedding** 을 사용합니다.
- 입력: `(batch,)` 형태의 정수 time step
- 출력: `(batch, dim)` 형태의 연속 벡터

### 4.2 ResNet Block & Weight Standardization

- U-Net의 기본 블록은 **GroupNorm + SiLU + Conv** 로 구성된 블록입니다.
- DDPM 구현에서는 Wide ResNet block 대신 **Weight Standardized Conv2d + GroupNorm** 을 사용해 학습 안정성을 높입니다.
- Time embedding은 각 ResNet block에 **scale/shift (FiLM-style)** 로 주입됩니다.


In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
    """
    타임스텝(t) 정보를 신경망이 이해할 수 있는 고차원 벡터로 변환합니다.
    Transformer의 positional encoding과 동일하게 sin, cos 함수를 사용합니다.
    """
    def __init__(self, dim):
        super().__init__()
        self.dim = dim # 결과 벡터의 차원 (보통 128, 256 등)

    def forward(self, time):
        # time: (batch_size,) 형태의 정수 타임스텝
        device = time.device
        half_dim = self.dim // 2

        # 주파수(frequencies) 계산: 각 차원마다 서로 다른 주기를 할당함 (1 -> 0.0001)
        freq = math.log(10000) / (half_dim - 1)
        freq = torch.exp(torch.arange(half_dim, device=device) * -freq)

        # 타임스텝 값에 주파수를 곱함 (batch, 1) * (1, half_dim) -> (batch, half_dim)
        # 예)
        # time[:,None] → [[10],[20],[30]] shape (3,1)
        # freq[None,:] → [[1.0,0.1,0.01,0.001]] shape (1,4)
        # args shape (3,4):
        # row0: [10, 1, 0.1, 0.01]
        # row1: [20, 2, 0.2, 0.02]
        # row2: [30, 3, 0.3, 0.03]
        args = time[:, None].float() * freq[None, :]

        # sin과 cos 값을 연결하여 하나의 임베딩 벡터 생성 (batch, dim)
        emb = torch.cat([args.sin(), args.cos()], dim=-1)
        return emb



class WeightStandardizedConv2d(nn.Conv2d):
    """
    Weight Standardization → kernel(weight) 정규화
    일반적인 Conv2d를 확장한 레이어입니다.
    각 출력 채널별 커널의 모든 Weight에 대해서 가중치(Weight)를 평균 0, 분산 1로 표준화하여
    Batch Size가 작을 때 학습을 안정화시키는 역할을 합니다.
    주로 GroupNorm과 같이 쓰입니다
    """
    def forward(self, x):
        # 수치적 안정성을 위한 아주 작은 값
        eps = 1e-5 if x.dtype == torch.float32 else 1e-3
        weight = self.weight # 컨볼루션 필터 자체의 가중치 (o, i, kH, kW)

        # 가중치의 평균과 분산을 출력 채널(o) 단위로 계산
        mean = reduce(weight, "o ... -> o 1 1 1", "mean")   # 채널마다 하나의 scalar, mean.shape == (o, 1, 1, 1)
        var = reduce(weight, "o ... -> o 1 1 1", partial(torch.var, unbiased=False))

        # 가중치 정규화 (Weight Standardization)
        normalized_weight = (weight - mean) / (var + eps).rsqrt()

        # 정규화된 가중치로 실제 컨볼루션 연산 수행
        return F.conv2d(
            x, normalized_weight, self.bias, self.stride,
            self.padding, self.dilation, self.groups,
        )


class Block(nn.Module):
    """
    기본 연산 단위: Convolution -> Group Normalization -> SiLU 활성화 함수
    scale_shift가 들어오면 타임스텝 정보를 특징 맵에 주입합니다.
    FiLM(Feature-wise Linear Modulation) or Adaptive Group Normalization (AdaGN)
    """
    def __init__(self, dim, dim_out, groups=8):
        super().__init__()
        self.proj = WeightStandardizedConv2d(dim, dim_out, 3, padding=1)
        self.norm = nn.GroupNorm(groups, dim_out) # 채널을 그룹으로 나눠 정규화 (배치 크기에 무관하게 안정적)
        self.act = nn.SiLU() # 최근 생성 모델에서 자주 쓰이는 매끄러운 활성화 함수

    def forward(self, x, scale_shift=None):
        x = self.proj(x)    # (B, dim, H, W) -> (B, dim_out, H, W)
        x = self.norm(x)    # shape 유지

        # 타임스텝 임베딩(scale_shift)이 있으면 특징 맵에 반영 (AdaIN과 유사한 방식)
        # 단순히 이미지 특징(x)에 타임스텝을 더하는 것이 아니라,
        # 타임스텝 정보에 따라 이미지 특징의 강도를 채널별로 조절(Scale)하고 편향을 수정(Shift)하는 것
        if exists(scale_shift):
            scale, shift = scale_shift  # scale:(B, C, 1, 1), shift:(B, C, 1, 1)
            # x = x * (1 + scale) + shift 공식을 통해 특징을 변형, 채널별 affine 변환 -> y = x⋅(1+s) + b
            x = x * (scale + 1) + shift

        x = self.act(x)
        return x


class ResnetBlock(nn.Module):
    """
    Diffusion 모델의 핵심 블록입니다.
    1. 입력 이미지에 타임스텝 정보를 섞어주고
    2. Residual 연결(잔차 연결)을 통해 깊은 층에서도 학습이 잘 되게 합니다.
    """
    def __init__(self, dim, dim_out, *, time_emb_dim=None, groups=8):
        super().__init__()
        # 타임스텝 정보를 가공하기 위한 작은 신경망(MLP)
        self.mlp = (
            nn.Sequential(
                nn.SiLU(),
                nn.Linear(time_emb_dim, dim_out * 2) # scale과 shift 두 값을 뽑기 위해 2배로 확장
            )
            if exists(time_emb_dim) else None
        )

        self.block1 = Block(dim, dim_out, groups=groups)
        self.block2 = Block(dim_out, dim_out, groups=groups)

        # 입력과 출력 채널이 다를 경우 맞춰주기 위한 1x1 Conv (잔차 연결용)
        self.res_conv = nn.Conv2d(dim, dim_out, 1) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb=None):
        scale_shift = None

        # 1. 타임스텝 정보를 현재 레이어의 채널 크기에 맞게 변환
        if exists(self.mlp) and exists(time_emb):
            time_emb = self.mlp(time_emb) # (batch, dim_out * 2)
            time_emb = rearrange(time_emb, "b c -> b c 1 1") # 이미지 공간 크기에 맞춰 차원 확장
            scale_shift = time_emb.chunk(2, dim=1) # (scale, shift)로 반씩 쪼갬

        # 2. 첫 번째 블록 (타임스텝 정보 주입) -> 이미지 특징에 지금은 t 단계다라는 정보가 각인
        h = self.block1(x, scale_shift=scale_shift)

        # 3. 두 번째 블록
        h = self.block2(h)

        # 4. 잔차 연결: 입력값 x를 결과에 더함 (정보 유실 방지)
        return h + self.res_conv(x)

## 5. Attention 모듈

DDPM의 U-Net 내부에는 **convolution block 사이사이에 attention** 이 섞여 있습니다.

- 이미지의 **글로벌 컨텍스트**를 보는 역할
- 두 가지 버전:
  - 일반 Multi-Head Self-Attention
  - Linear Attention (sequence 길이에 대해 O(N) scaling)

아래는 두 attention 모듈과, attention 전에 GroupNorm을 적용하는 `PreNorm` 래퍼입니다.


In [ ]:
class Attention(nn.Module):
    """
    표준적인 Multi-Head Self-Attention입니다.
    이미지의 모든 픽셀 쌍 사이의 유사도를 계산하므로 정확하지만,
    해상도가 커지면 계산량이 급격히(제곱으로) 늘어납니다.
    Attn(Q,K,V) = softmax(Q⊤ K)V
    """
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.scale = dim_head ** -0.5 # Softmax 전 수치 안정성을 위한 스케일링 상수 (1/sqrt(d_k))
        self.heads = heads
        hidden_dim = dim_head * heads

        # 1x1 Conv를 사용해 입력 채널을 Q, K, V 세 덩어리로 한꺼번에 생성, nn.Conv2d(in_chs, out_chs, kernel_size,...)
        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)
        # 최종 결과를 다시 원래 채널 크기로 돌려주는 1x1 Conv
        self.to_out = nn.Conv2d(hidden_dim, dim, 1)

    def forward(self, x):
        b, c, h, w = x.shape

        # 1. Q, K, V 생성 및 분리
        # (B, dim, H, W) -> (B, hidden_dim * 3, H, W) -> 3개의 (B, hidden_dim, H, W)
        qkv = self.to_qkv(x).chunk(3, dim=1)

        # 2. 데이터를 'Head' 별로 쪼개고 2D 이미지를 1D 시퀀스로 펼침 (Flatten)
        # (B, hidden_dim, H, W) -> (B, heads, dim_head, H*W)
        q, k, v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )

        # 3. Attention Score 계산 (Q와 K의 내적)
        q = q * self.scale
        # sim shape: (B, heads, HW, HW) -> 모든 픽셀 i와 모든 픽셀 j 사이의 관계를 행렬로 표현
        sim = torch.einsum("b h d i, b h d j -> b h i j", q, k) # (B, heads, N, N), where N=H*W

        # 수치적 안정성을 위해 최대값을 빼줌 (Softmax 폭주 방지)
        # 가장 큰 값은 항상 0이 되고, 나머지는 음수가 됨
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        # 4. Attention Map 생성 (가중치 합이 1이 되도록)
        attn = sim.softmax(dim=-1)

        # 5. Value에 Attention 가중치 적용
        # (B, heads, HW, HW) * (B, heads, head_dim, HW) -> (B, heads, HW, head_dim)
        out = torch.einsum("b h i j, b h d j -> b h i d", attn, v)

        # 6. 다시 2D 이미지 형태로 복원 (Reshape)
        # (B, heads, HW, head_dim) -> (B, heads*head_dim, H, W)
        out = rearrange(out, "b h (x y) d -> b (h d) x y", x=h, y=w)
        return self.to_out(out)



class LinearAttention(nn.Module):
    """
    Attention의 계산량을 줄인 효율적인 버전입니다.
    표준 Attention이 (픽셀수 * 픽셀수)만큼 계산할 때, 이 방식은 (픽셀수 * 채널수)만큼만 계산합니다.
    고해상도 특징 맵(U-Net의 앞부분)에서 주로 사용됩니다.
      - N×N 행렬을 만들지 말자 (O(N^2)
      - 결합 법칙으로 계산 순서를 바꾸자 (O(Nd)
    Attn(Q,K,V) ≈ ϕ(Q)(ϕ(K)^T V)
      - ϕ(Q)=softmax(Q over feature dim)
      - ϕ(K)=softmax(K over token dim)    
    """
    def __init__(self, dim, heads=4, dim_head=32):
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        hidden_dim = dim_head * heads

        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias=False)
        self.to_out = nn.Sequential(
            nn.Conv2d(hidden_dim, dim, 1),
            nn.GroupNorm(1, dim), # 정규화 추가로 학습 안정화
        )

    def forward(self, x):
        # x shape: (batch, dim, height, width) 
        b, c, h, w = x.shape

        # 1. Q, K, V 생성 (1x1 Conv 사용)
        # qkv shape: 3개의 (batch, hidden_dim, height, width) 덩어리
        qkv = self.to_qkv(x).chunk(3, dim=1)

        # 2. 헤드 분리 및 2D 이미지를 1D 시퀀스로 변환 (Flatten)
        # q, k, v 각각의 shape: (batch, heads, dim_head, pixels)
        q, k, v = map(
            lambda t: rearrange(t, "b (h c) x y -> b h c (x y)", h=self.heads), qkv
        )

        # 3. Q, K 정규화 (Linear Attention의 핵심)
        # q shape: (b, h, d, n) - 특징(d) 차원에 대해 softmax
        # k shape: (b, h, d, n) - 픽셀(n) 차원에 대해 softmax
        q = q.softmax(dim=-2)
        k = k.softmax(dim=-1)
        q = q * self.scale

        # 4. K와 V를 먼저 결합하여 '글로벌 컨텍스트' 생성 (결합 법칙 활용)
        # 연산: (b, h, dim_head, pixels) @ (b, h, dim_head, pixels)^T
        # 결과 shape: (batch, heads, dim_head, dim_head)
        # ★ 중요: 픽셀 수가 사라지고 특징 차원만 남음! (계산량 급감)
        context = torch.einsum("b h d n, b h e n -> b h d e", k, v)

        # 5. 생성된 컨텍스트 정보를 Q에 적용
        # 연산: (b, h, dim_head, dim_head) @ (b, h, dim_head, pixels)
        # 결과 shape: (batch, heads, dim_head, pixels)
        out = torch.einsum("b h d e, b h d n -> b h e n", context, q)

        # 6. 이미지 공간 형태로 다시 복원 (Reshape)
        # out shape: (batch, hidden_dim, height, width)
        out = rearrange(out, "b h c (x y) -> b (h c) x y", h=self.heads, x=h, y=w)

        # 7. 최종 출력 투영 및 정규화
        return self.to_out(out)


class PreNorm(nn.Module):
    """
    '정규화 후 연산' 방식을 구현한 래퍼(Wrapper)입니다.
    Attention이나 FeedForward 레이어 직전에 GroupNorm을 배치하여 그래디언트 흐름을 돕고 깊은 신경망의 학습을 용이하게 합니다.
    timestep t마다 feature 분포가 완전히 다름 -> PreNorm으로 입력 분포를 먼저 정규화
    """
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.GroupNorm(1, dim) # 채널 전체를 하나의 그룹으로 정규화 (LayerNorm과 유사)
        self.fn = fn # 감싸질 실제 함수 (Attention 등)

    def forward(self, x):
        # 1. 먼저 정규화를 수행하고
        # 2. 그 결과를 본래의 함수(fn)에 전달함
        return self.fn(self.norm(x))

## 6. U-Net 구현

이제까지 만든 구성 요소를 써서 **U-Net 기반 노이즈 예측 네트워크**를 정의합니다.

### 6.1 입력 / 출력 형태

- **입력**
  - $x_t$: `(batch, channels, height, width)` 형태의 noised image
  - $t$: `(batch,)` 형태의 time step (정수)
- **출력**
  - $\hat{\epsilon}_\theta(x_t, t)$: 입력에 추가된 **노이즈 텐서**와 동일한 shape

### 6.2 네트워크 구조 개략

1. **초기 Conv**로 채널 수를 base dimension으로 맞춤
2. **여러 단계의 Downsampling**:
   - `ResnetBlock` $\rightarrow$ `ResnetBlock` $\rightarrow$ `LinearAttention` $\rightarrow$ `Downsample`
3. **중간(Middle) block**:
   - `ResnetBlock` $\rightarrow$ `Attention` $\rightarrow$ `ResnetBlock`
4. **여러 단계의 Upsampling**:
   - `ResnetBlock` $\rightarrow$ `ResnetBlock` $\rightarrow$ `LinearAttention` $\rightarrow$ `Upsample`
5. **최종 ResNet block + $1 \times 1$ Conv**로 출력 채널 수를 맞춤

> **Skip connection**은 down path에서 저장한 feature들을 up path에서 `concat` 하는 방식으로 사용합니다.

In [ ]:
class Unet(nn.Module):
    def __init__(
        self,
        dim,                         # 기본 채널 차원 (예: 64)
        init_dim=None,               # 초기 컨볼루션 출력 차원
        out_dim=None,                # 최종 출력 채널 수 (보통 입력 채널과 동일)
        dim_mults=(1, 2, 4, 8),      # 단계별 채널 배수 (dim * m)
        channels=3,                  # 입력 이미지 채널 (RGB=3)
        self_condition=False,        # Self-conditioning 사용 여부
        resnet_block_groups=8,       # GroupNorm에서 사용할 그룹 수
    ):
        super().__init__()

        # 1. 채널 설정 및 초기화
        self.channels = channels
        self.self_condition = self_condition
        # self_condition이 True면 이전 단계 예측값을 입력에 합치므로 채널이 2배가 됨
        input_channels = channels * (2 if self_condition else 1)

        init_dim = default(init_dim, dim)
        self.init_conv = nn.Conv2d(input_channels, init_dim, 1, padding=0)

        # 2. 단계별 매추차원 계산 (예: [64, 64, 128, 256, 512])
        dims = [init_dim, *map(lambda m: dim * m, dim_mults)]
        # 각 단계의 입력/출력 채널 쌍 생성 (예: [(64, 64), (64, 128), (128, 256), (256, 512)])
        in_out = list(zip(dims[:-1], dims[1:]))

        # ResnetBlock 설정을 고정 (groups 파라미터 미리 지정)
        block = partial(ResnetBlock, groups=resnet_block_groups)

        # 3. 타임 임베딩 (Time Embedding) 생성부
        # 숫자인 t를 고차원 벡터로 바꾼 후 MLP를 통해 특징 추출
        time_dim = dim * 4
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(dim), # 1. 숫자를 사인/코사인 파동 벡터로 변환
            nn.Linear(dim, time_dim),          # 2. 차원 확장
            nn.GELU(),                         # 3. 활성화 함수
            nn.Linear(time_dim, time_dim),     # 4. 최종 임베딩 벡터 완성
        )

        # 4. Downsampling Path (인코더)
        self.downs = nn.ModuleList([])
        num_resolutions = len(in_out)

        for i, (dim_in, dim_out) in enumerate(in_out):
            is_last = i == (num_resolutions - 1)
            self.downs.append(
                nn.ModuleList(
                    [
                        block(dim_in, dim_in, time_emb_dim=time_dim), # ResNet 블록 1
                        block(dim_in, dim_in, time_emb_dim=time_dim), # ResNet 블록 2
                        Residual(PreNorm(dim_in, LinearAttention(dim_in))), # 효율적인 선형 어텐션
                        # 마지막 단계가 아니면 해상도를 줄이고(Downsample), 맞으면 채널만 변경
                        Downsample(dim_in, dim_out) if not is_last else nn.Conv2d(dim_in, dim_out, 3, padding=1),
                    ]
                )
            )

        # 5. Middle Block (최하단 보틀넥)
        mid_dim = dims[-1]
        self.mid_block1 = block(mid_dim, mid_dim, time_emb_dim=time_dim)
        self.mid_attn = Residual(PreNorm(mid_dim, Attention(mid_dim))) # 여기서 정밀한 표준 어텐션 사용
        self.mid_block2 = block(mid_dim, mid_dim, time_emb_dim=time_dim)

        # 6. Upsampling Path (디코더)
        self.ups = nn.ModuleList([])
        for i, (dim_in, dim_out) in enumerate(reversed(in_out)):
            is_last = i == (len(in_out) - 1)
            self.ups.append(
                nn.ModuleList(
                    [
                        # Skip connection으로 인해 채널이 dim_out + dim_in으로 들어옴 (Concat 결과)
                        block(dim_out + dim_in, dim_out, time_emb_dim=time_dim),
                        block(dim_out + dim_in, dim_out, time_emb_dim=time_dim),
                        Residual(PreNorm(dim_out, LinearAttention(dim_out))),
                        # 마지막 단계가 아니면 해상도를 키움(Upsample)
                        Upsample(dim_out, dim_in) if not is_last else nn.Conv2d(dim_out, dim_in, 3, padding=1),
                    ]
                )
            )

        # 7. 최종 출력층
        self.out_dim = default(out_dim, channels)
        self.final_res_block = block(dim * 2, dim, time_emb_dim=time_dim)
        self.final_conv = nn.Conv2d(dim, self.out_dim, 1)

    def forward(self, x, time, x_self_cond=None):
        # [0] Self-conditioning 처리: 이전 단계 예측값을 현재 입력 채널에 결합
        if self.self_condition:
            x_self_cond = default(x_self_cond, lambda: torch.zeros_like(x))
            x = torch.cat([x_self_cond, x], dim=1)

        # [1] 초기 컨볼루션 및 원본 특징 복사 (최종 Skip connection용)
        x = self.init_conv(x)
        r = x.clone()

        # [2] 타임 임베딩 계산: (batch,) -> (batch, time_dim)
        t = self.time_mlp(time)

        # Skip Connection 데이터를 담을 리스트
        h = []

        # [3] Down path (이미지를 작게 줄이며 특징 추출)
        for block1, block2, attn, downsample in self.downs:
            x = block1(x, t)
            h.append(x)          # Skip connection을 위해 특징 저장 1

            x = block2(x, t)
            h.append(x)          # Skip connection을 위해 특징 저장 2

            x = attn(x)          # 글로벌 정보 참조
            x = downsample(x)    # 해상도 감소 (H, W -> H/2, W/2)

        # [4] Middle (보틀넥: 가장 추상화된 특징 연산)
        x = self.mid_block1(x, t)
        x = self.mid_attn(x)
        x = self.mid_block2(x, t)

        # [5] Up path (이미지를 다시 키우며 Skip Connection 결합)
        for block1, block2, attn, upsample in self.ups:
            # h.pop()으로 가장 나중에 저장된(가장 작은 해상도) 특징부터 꺼내서 결합
            x = torch.cat([x, h.pop()], dim=1) # 채널 결합 (Concat)
            x = block1(x, t)

            x = torch.cat([x, h.pop()], dim=1) # 채널 결합 (Concat)
            x = block2(x, t)

            x = attn(x)
            x = upsample(x)      # 해상도 복구 (H, W -> 2H, 2W)

        # [6] Final (원본 크기 복구 및 최종 노이즈 예측)
        x = torch.cat([x, r], dim=1)       # 최초의 특징(r)까지 결합
        x = self.final_res_block(x, t)
        return self.final_conv(x)          # (B, out_dim, H, W)

## 7. Forward Diffusion Process 정의

### 7.1 Beta Schedule

- Forward 과정(노이즈 주입 과정)은 다음과 같은 형태를 가집니다.

$$q(x_t \mid x_{t-1}) = \mathcal{N}(\sqrt{1 - \beta_t} x_{t-1},\; \beta_t \mathbf{I})$$

- 여기서 $\beta_t$는 **시간에 따라 증가하는 노이즈의 분산**입니다.
- DDPM 논문에서는 주로 **linear schedule**을 사용했습니다.
- 추가로 quadratic, sigmoid, cosine schedule 등도 사용할 수 있습니다.

### 7.2 $q(x_t \mid x_0)$의 닫힌 형태 (Closed-form)

- 마르코프 체인(Markov Chain)의 성질을 이용해 식을 정리하면, 중간 단계를 거치지 않고 $x_0$에서 임의의 시점 $t$의 이미지를 바로 얻을 수 있습니다.

$$\bar{\alpha}_t = \prod_{s=1}^t (1 - \beta_s)$$

$$q(x_t \mid x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) \mathbf{I})$$



- 이 식이 있어, 학습 시에는 $x_0$에서 **한 번에 $x_t$를 샘플링**할 수 있어 매우 효율적입니다.

In [ ]:
import torch
import torch.nn.functional as F

# -------------------------------------------------------------------
# 1. 다양한 Beta Schedule 함수들
# 목적: 시간이 흐름(t)에 따라 노이즈를 얼마나 강하게 주입할지 결정합니다.
# -------------------------------------------------------------------

def cosine_beta_schedule(timesteps, s=0.008):
    """
    OpenAI의 'Improved DDPM' 논문에서 제안된 코사인 스케줄입니다.
    Linear 스케줄보다 노이즈가 더 천천히 주입되어 이미지의 세부 정보를 더 잘 보존합니다.
    βt = 1 − ( αˉt / αˉt−1 )
    """
    steps = timesteps + 1
    # 0부터 timesteps까지 일정한 간격으로 생성
    x = torch.linspace(0, timesteps, steps)

    # alphas_cumprod(알파 바)를 코사인 함수로 먼저 정의합니다.
    # αˉt = cos^2((t/T+s)/(1+s) * π/2)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * torch.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0] # 시작 값을 1로 정규화

    # betas = 1 - (현재 알파바 / 이전 알파바) 수식을 통해 역산합니다.
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])

    # 수치적 안정성을 위해 최소/최대 값을 제한합니다.
    return torch.clip(betas, 0.0001, 0.9999)

def linear_beta_schedule(timesteps):
    """가장 기본적인 선형 스케줄. 노이즈 강도가 t에 따라 일정하게 증가합니다."""
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

def quadratic_beta_schedule(timesteps):
    """노이즈가 초반에는 천천히, 후반에는 급격하게 증가하는 제곱 스케줄입니다."""
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start**0.5, beta_end**0.5, timesteps) ** 2

def sigmoid_beta_schedule(timesteps):
    """S자 곡선 형태로 노이즈를 주입합니다. 초반과 후반은 완만하고 중간은 급격합니다."""
    beta_start = 0.0001
    beta_end = 0.02
    betas = torch.linspace(-6, 6, timesteps)
    return torch.sigmoid(betas) * (beta_end - beta_start) + beta_start

# -------------------------------------------------------------------
# 2. Diffusion 수식에 필요한 계수(Constants) 계산부
# -------------------------------------------------------------------

timesteps = 300 # 전체 타임스텝 (T)
betas = linear_beta_schedule(timesteps=timesteps) # Beta_t

# alpha_t = 1 - beta_t
alphas = 1.0 - betas

# alpha_cumprod (알파 바_t): t시점까지의 모든 alpha를 곱한 값
# x_0에서 바로 x_t를 샘플링할 때 사용됩니다.
alphas_cumprod = torch.cumprod(alphas, dim=0)

# alpha_cumprod_prev (알파 바_{t-1}): 이전 단계의 누적 곱
# 마지막 값을 제외하고 맨 앞에 1.0을 패딩하여 (T,) 크기를 유지합니다.
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

# -------------------------------------------------------------------
# 3. Forward/Reverse 과정에서 반복 사용될 수식들 미리 계산
# -------------------------------------------------------------------

# 1 / sqrt(alpha_t): 샘플링 시 x_t에서 노이즈 성분을 뺄 때 사용
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)

# sqrt(alpha_bar_t): x_0의 계수 (Mean 성분)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)

# sqrt(1 - alpha_bar_t): 노이즈(epsilon)의 계수 (Variance 성분)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

# posterior_variance: 역과정(Reverse Process)에서 사용하는 분산 식
# q(x_{t-1} | x_t, x_0)의 분산을 계산할 때 쓰입니다.
posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)

# -------------------------------------------------------------------
# 4. 유틸리티 함수
# -------------------------------------------------------------------
# 1차원 리스트에서 필요한 값만 뽑아서,
# 이미지 텐서와 바로 곱할 수 있도록 빈 차원(1)을 붙여주는 '어댑터' 같은 함수
def extract(a, t, x_shape):
    """
    1차원 텐서(a)에서 현재 배치 멤버들의 타임스텝(t)에 해당하는 값만 골라내어
    이미지 텐서 연산이 가능하도록 차원을 맞춰줍니다.

    Args:
        a: 미리 계산된 계수 리스트 (예: sqrt_alphas_cumprod)
        t: 현재 배치의 타임스텝들 (Batch_size,)
        x_shape: 현재 이미지의 shape (Batch, Channel, H, W)

    Returns:
        (Batch, 1, 1, 1) 형태의 텐서
    """
    batch_size = t.shape[0]
    
    # t에 해당하는 인덱스 값들을 추출 : [ a[t[0]], a[t[1]], ... ]
    out = a.gather(-1, t.cpu())
    
    # 이미지 차원(4D)에 맞춰 뒤에 1들을 추가: (B,) -> (B, 1, 1, 1)
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

## 8. Forward Diffusion 시각화 (예시 이미지)

DDPM이 실제로 어떤 식으로 이미지를 **노이즈로 바꿔가는지** 직관적으로 보기 위해 간단한 예시 이미지를 사용합니다.

1. 인터넷에서 샘플 이미지를 하나 불러온 뒤 (예: COCO 예시)
2. 이미지를 $[-1, 1]$ 범위의 텐서로 변환 (모델의 입력 규격에 맞춤)
3. 여러 타임스텝 $t$에 대해 $x_t \sim q(x_t \mid x_0)$를 샘플링
4. 각 단계의 이미지를 플롯(Plot)으로 비교하여 노이즈 강도 변화 관찰



> **Notice:** Diffusion의 기본 개념(정보의 점진적 파괴)을 직관적으로 이해할 수 있음.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
import torch
from torchvision.transforms import Compose, ToTensor, Lambda, CenterCrop, Resize, ToPILImage

# -------------------------------------------------------------------
# 1. 샘플 이미지 준비 및 전처리 (Preprocessing)
# -------------------------------------------------------------------

# 1) 웹에서 이미지 불러오기 (COCO 데이터셋의 고양이 이미지)
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image_size = 128

# 2) 전처리 파이프라인 구성
# Diffusion 모델은 일반적으로 [-1, 1] 범위의 데이터를 사용합니다.
transform = Compose([
    Resize(image_size),           # 짧은 쪽을 128로 조절
    CenterCrop(image_size),       # 중앙을 128x128로 크롭
    ToTensor(),                   # PIL 이미지를 [0, 1] 범위의 텐서로 변환 (C, H, W)
    Lambda(lambda t: (t * 2) - 1),# [0, 1] 범위를 [-1, 1] 범위로 선형 변환
])

# x_start: 원본 이미지(x0). 모델 입력용 배치를 위해 (1, 3, 128, 128)로 확장
x_start = transform(image).unsqueeze(0)

# 3) 역전처리 파이프라인 (모델 출력인 텐서를 다시 눈으로 볼 수 있는 이미지로)
reverse_transform = Compose([
    Lambda(lambda t: (t + 1) / 2),           # [-1, 1] -> [0, 1]
    Lambda(lambda t: t.permute(1, 2, 0)),    # (C, H, W) -> (H, W, C) 순서 변경
    Lambda(lambda t: (t * 255.).clamp(0, 255)), # [0, 1] -> [0, 255] 정규화 및 클리핑
    Lambda(lambda t: t.numpy().astype(np.uint8)), # 텐서를 넘파이(uint8) 배열로 변환
    ToPILImage(),                            # 배열을 다시 PIL 이미지 객체로 변환
])

# -------------------------------------------------------------------
# 2. Forward Diffusion 수식 구현 (q_sample)
# -------------------------------------------------------------------

def q_sample(x_start, t, noise=None):
    """
    수식: q(x_t | x_0) = sqrt(alpha_bar_t) * x_0 + sqrt(1 - alpha_bar_t) * epsilon
    이 수식은 x_0에서 중간 단계 없이 즉시 x_t를 샘플링하게 해줍니다.
    """
    if noise is None:
        # 가우시안 노이즈(epsilon) 생성
        noise = torch.randn_like(x_start)

    # 미리 계산해둔 계수 리스트에서 현재 t에 해당하는 계수를 (B, 1, 1, 1)로 추출
    sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x_start.shape
    )

    # 수식 그대로 계산 (원본의 강도는 줄이고, 노이즈의 강도는 키우고)
    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

def get_noisy_image(x_start, t):
    """특정 타임스텝 t의 노이즈 섞인 이미지를 시각화 가능한 이미지로 반환"""
    x_noisy = q_sample(x_start, t=t)
    return reverse_transform(x_noisy.squeeze()) # 배치 차원을 제거하고 이미지로 변환

# -------------------------------------------------------------------
# 3. 시각화 함수 (여러 단계를 한눈에 비교)
# -------------------------------------------------------------------

def plot(imgs, with_orig=False, row_title=None, **imshow_kwargs):
    """이미지 리스트를 격자 형태로 출력하는 함수"""
    if not isinstance(imgs[0], list):
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + (1 if with_orig else 0)

    fig, axs = plt.subplots(
        figsize=(2 * num_cols, 2 * num_rows),
        nrows=num_rows, ncols=num_cols, squeeze=False,
    )

    for row_idx, row in enumerate(imgs):
        row = [image] + row if with_orig else row # 원본 포함 여부
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title="Original")
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])
    plt.tight_layout()

# 실행: 타임스텝 0, 50, 100, 150, 250에서의 변화 시각화
# t가 커질수록(오른쪽으로 갈수록) 이미지가 파괴되어 노이즈로 변하는 과정을 확인하세요.
plot(
    [get_noisy_image(x_start, torch.tensor([t])) for t in [0, 50, 100, 150, 250]],
    with_orig=True
)

## 9. 학습 손실 함수 정의

DDPM의 학습 목표는 **신경망($\theta$)이 이미지에 추가된 노이즈 $\epsilon$을 얼마나 정확하게 예측하는가**를 측정하는 것입니다.

### 학습 알고리즘 단계

1. 데이터셋에서 깨끗한 이미지 $x_0$를 샘플링합니다.
2. 전체 타임스텝 중 임의의 시점 $t \sim \text{Uniform}(1, \dots, T)$를 샘플링합니다.
3. 이미지에 섞을 가우시안 노이즈 $\epsilon \sim \mathcal{N}(0, \mathbf{I})$를 생성합니다.
4. $x_0$와 $\epsilon$을 사용하여 노이즈가 섞인 이미지 $x_t$를 계산합니다.
   $$x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1-\bar{\alpha}_t} \epsilon$$
5. U-Net 네트워크에 $x_t$와 타임스텝 $t$를 입력하여 예측 노이즈 $\hat{\epsilon}_\theta(x_t, t)$를 얻습니다.
6. 실제 노이즈 $\epsilon$과 예측된 노이즈 $\hat{\epsilon}_\theta$ 사이의 **MSE(L2)** 또는 **Huber Loss**를 최소화합니다.
   $$\mathcal{L} = \| \epsilon - \hat{\epsilon}_\theta(x_t, t) \|^2$$



> 이 과정을 코드로 구현한 것이 `p_losses` 함수이며, 모델은 이 손실 함수를 통해 이미지에서 노이즈 성분만 골라내는 방법을 배우게 됩니다.

In [ ]:
import torch.nn.functional as F

def p_losses(denoise_model, x_start, t, noise=None, loss_type="l1"):
    """
    Diffusion 모델을 학습시키기 위한 손실 함수(Loss Function) 계산 함수입니다.
    네트워크(U-Net)가 이미지에 섞인 노이즈를 얼마나 잘 찾아내는지 측정합니다.
    """

    # 1. 실제 노이즈(Ground Truth Noise) 생성
    # 이미지와 동일한 크기의 가우시안 노이즈(epsilon)를 만듭니다.
    if noise is None:
        noise = torch.randn_like(x_start)

    # 2. Forward Diffusion: x_0(원본)에서 x_t(노이즈 섞인 상태) 계산
    # 앞서 정의한 q_sample 함수를 사용하여 t 시점의 노이즈 이미지 x_t를 얻습니다.
    # 수식: x_t = sqrt(alpha_bar_t) * x_start + sqrt(1 - alpha_bar_t) * noise
    x_noisy = q_sample(x_start=x_start, t=t, noise=noise)

    # 3. 모델의 예측
    # U-Net 모델(denoise_model)에 '노이즈 이미지(x_noisy)'와 '타임스텝(t)'을 입력합니다.
    # 모델은 "이 이미지의 이 타임스텝에는 어떤 노이즈가 섞여 있어?"라는 질문에 답합니다.
    predicted_noise = denoise_model(x_noisy, t)

    # 4. 손실(Loss) 계산
    # 실제로 섞은 노이즈(noise)와 모델이 예측한 노이즈(predicted_noise) 사이의 오차를 구합니다.
    if loss_type == "l1":
        # L1 Loss: 오차의 절댓값 평균 (이상치에 상대적으로 강건함)
        loss = F.l1_loss(noise, predicted_noise)
    elif loss_type == "l2":
        # L2 Loss (MSE): 오차의 제곱 평균 (표준적인 DDPM 학습 방식)
        loss = F.mse_loss(noise, predicted_noise)
    elif loss_type == "huber":
        # Huber Loss (Smooth L1): L1과 L2의 장점을 섞은 방식 (학습 안정화에 도움)
        loss = F.smooth_l1_loss(noise, predicted_noise)
    else:
        raise NotImplementedError("지원하지 않는 loss_type입니다.")

    return loss

## 10. Fashion-MNIST 데이터셋 로드 및 전처리

실습에서는 **Hugging Face `fashion_mnist` 데이터셋**을 사용합니다.

- 28x28 grayscale 이미지 (의류/신발 등 10 클래스)
- 여기서는 **레이블은 사용하지 않고**, 오직 이미지 분포만 학습합니다.
- 이미지를 \([-1, 1]\) 범위의 텐서로 변환하고, 간단한 augmentation으로 random horizontal flip을 적용합니다.


In [ ]:
# -------------------------------------------------------------------
# 1. Fashion-MNIST 데이터셋 로드
# -------------------------------------------------------------------
# Hugging Face의 datasets 라이브러리를 사용하여 데이터를 불러옵니다.
# 이 데이터셋은 28x28 해상도의 흑백 의류 이미지들로 구성되어 있습니다.
dataset = load_dataset("fashion_mnist")

# 하이퍼파라미터 설정
image_size = 28  # Fashion-MNIST의 기본 크기
channels = 1    # 흑백(Grayscale) 이미지이므로 채널은 1
batch_size = 128

# -------------------------------------------------------------------
# 2. 이미지 전처리(Augmentation & Normalization) 정의
# -------------------------------------------------------------------
transform = transforms.Compose([
    # 좌우 반전을 무작위로 적용하여 데이터의 다양성을 확보합니다.
    transforms.RandomHorizontalFlip(),

    # PIL 이미지를 PyTorch 텐서로 변환하며 값을 [0, 1] 범위로 정규화합니다.
    transforms.ToTensor(),

    # Diffusion 모델의 안정적인 학습을 위해 [0, 1] 범위를 [-1, 1] 범위로 변환합니다.
    # 수식: (0 * 2) - 1 = -1, (1 * 2) - 1 = 1
    transforms.Lambda(lambda t: (t * 2) - 1),
])

# -------------------------------------------------------------------
# 3. 데이터셋 변환 함수 정의 (Hugging Face 전용)
# -------------------------------------------------------------------
def transform_examples(examples):
    """
    Hugging Face 데이터셋의 각 샘플에 전처리를 적용하는 함수입니다.
    """
    # 1. 원본 'image' 열에서 이미지를 꺼내 'L'(흑백) 모드로 확실히 변환 후 전처리 적용
    # 2. 전처리된 결과를 'pixel_values'라는 새로운 키에 리스트 형태로 저장
    #    - 변환 전 : {"image": [PIL객체1, PIL객체2, ...], "label": [9, 0, ...]}
    #    - 변환 후 : {"pixel_values": [텐서1, 텐서2, ...], "image": [PIL객체1, PIL객체2, ...], "label": [9, 0, ...]}
    examples["pixel_values"] = [transform(image.convert("L")) for image in examples["image"]]

    # 메모리 절약을 위해 원본 이미지 데이터는 삭제합니다.
    del examples["image"]
    return examples

# 데이터셋에 변환 함수를 등록하고, 사용하지 않는 'label' 열은 제거합니다.
# (이 실습은 비지도 학습인 Diffusion이므로 레이블이 필요 없습니다.)
transformed_dataset = dataset.with_transform(transform_examples).remove_columns("label")

# -------------------------------------------------------------------
# 4. DataLoader 구축 (Batch 생성)
# -------------------------------------------------------------------
# 실제 학습 루프에서 데이터를 효율적으로 공급하기 위해 DataLoader를 사용합니다.
dataloader = DataLoader(
    transformed_dataset["train"],
    batch_size=batch_size,
    shuffle=True,  # 매 에포크마다 데이터를 섞어 학습 성능을 높입니다.
    drop_last=True # 배치 사이즈에 맞지 않는 마지막 자투리 데이터는 버립니다.
)

# 데이터 로드가 잘 되었는지 첫 번째 배치를 확인해봅니다.
batch = next(iter(dataloader))
# 결과 출력: (배치 사이즈, 채널, 높이, 너비) -> (128, 1, 28, 28)
print(f"Batch shape: {batch['pixel_values'].shape}")

## 11. Reverse Diffusion (Sampling) 정의

이제 학습이 끝났다고 가정하고, **아무것도 없는 노이즈(Pure Noise)에서 시작해서 이미지를 생성하는 과정**을 정의합니다.

### 11.1 한 스텝 샘플링: `p_sample`

현재 단계의 이미지 $x_t$에서 이전 단계의 이미지 $x_{t-1}$를 추론하는 과정은 다음과 같은 확률 분포를 따릅니다.

$$p_\theta(x_{t-1} \mid x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \sigma_t^2 \mathbf{I})$$

- 여기서 $\mu_\theta$는 네트워크가 예측한 **노이즈($\hat{\epsilon}_\theta$)**를 이용하여 계산된 평균입니다.
- $\sigma_t^2$는 미리 정의된 노이즈의 분산입니다.
- 모델은 한 번에 이미지를 완성하는 것이 아니라, $x_t$에서 노이즈 성분을 살짝 제거하여 조금 더 선명한 $x_{t-1}$을 만듭니다.

### 11.2 전체 샘플링 루프 (Reverse Process)

전체 생성 과정은 다음과 같은 순서로 진행됩니다.

1. **시작**: 완전히 무작위인 노이즈 $x_T \sim \mathcal{N}(0, \mathbf{I})$에서 시작합니다.
2. **반복**: $t = T-1, T-2, \dots, 0$까지 거꾸로 거슬러 올라가며 `p_sample`을 적용합니다.
3. **완성**: 최종적으로 노이즈가 모두 제거된 깨끗한 이미지 샘플 $x_0$를 얻습니다.



> **핵심 포인트:** 학습할 때는 노이즈를 **입히는 법**($\epsilon$)을 배웠지만, 생성할 때는 배운 지식을 활용해 노이즈를 **걷어내는 법**을 실행하는 것입니다.

In [ ]:
import torch
from tqdm.auto import tqdm

# -------------------------------------------------------------------
# 1. p_sample: 현재 단계(t)에서 이전 단계(t-1)의 이미지를 추론
# -------------------------------------------------------------------
@torch.no_grad() # 샘플링 시에는 그래디언트 계산이 필요 없으므로 메모리를 절약합니다.
def p_sample(model, x, t, t_index):
    """
    노이즈가 섞인 이미지 x에서 모델이 예측한 노이즈를 빼서
    조금 더 깨끗한 이전 단계의 이미지를 계산합니다.
    """
    # 현재 타임스텝 t에 해당하는 미리 계산된 계수들을 가져옵니다.
    betas_t = extract(betas, t, x.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x.shape
    )
    sqrt_recip_alphas_t = extract(sqrt_recip_alphas, t, x.shape)

    # [수식 구현] DDPM 논문의 Equation 11: 모델이 예측한 평균(mu) 계산
    # '현재 이미지'에서 '모델이 예측한 노이즈' 성분을 적절한 비율로 빼주는 과정입니다.
    # model(x, t)는 U-Net이 예측한 노이즈(epsilon_theta)입니다.
    model_mean = sqrt_recip_alphas_t * (
        x - betas_t * model(x, t) / sqrt_one_minus_alphas_cumprod_t
    )

    if t_index == 0:
        # 마지막 단계(t=0)라면 평균값을 그대로 반환하여 최종 이미지를 얻습니다.
        return model_mean
    else:
        # t > 0 이라면 계산된 평균에 약간의 무작위 노이즈를 더해줍니다.
        # 이 과정이 있어야 Langevin Dynamics처럼 확률적으로 데이터 분포를 찾아갑니다.
        posterior_variance_t = extract(posterior_variance, t, x.shape)
        noise = torch.randn_like(x) # epsilon ~ N(0, I)

        # 평균 + 표준편차 * 노이즈
        return model_mean + torch.sqrt(posterior_variance_t) * noise

# -------------------------------------------------------------------
# 2. p_sample_loop: T부터 0까지 반복하여 이미지 생성
# -------------------------------------------------------------------
@torch.no_grad()
def p_sample_loop(model, shape):
    """
    완전한 노이즈에서 시작하여 타임스텝을 거꾸로 거슬러 올라가며
    최종 이미지를 생성하는 전체 루프입니다.
    """
    device = next(model.parameters()).device
    b = shape[0] # 배치 사이즈

    # 1. 시작점: 완전한 가우시안 노이즈 x_T 생성 (Pure Noise)
    img = torch.randn(shape, device=device)
    imgs = [] # 변화 과정을 저장할 리스트

    # 2. 역과정 반복: T-1부터 0까지 거꾸로 진행 (Reverse Process)
    # tqdm을 사용하여 샘플링 진행 상황을 시각적으로 표시합니다.
    for i in tqdm(reversed(range(0, timesteps)), desc="sampling loop time step", total=timesteps):
        # p_sample 함수를 호출하여 한 단계 더 깨끗한 이미지를 얻음
        img = p_sample(
            model,
            img,
            torch.full((b,), i, device=device, dtype=torch.long), # 현재 타임스텝 t를 텐서로 전달
            i,
        )
        # 나중에 애니메이션이나 시각화를 위해 결과 저장
        imgs.append(img.cpu().numpy())

    return imgs

# -------------------------------------------------------------------
# 3. sample: 사용자용 최종 호출 함수
# -------------------------------------------------------------------
@torch.no_grad()
def sample(model, image_size, batch_size=16, channels=1):
    """
    외부에서 간편하게 이미지를 생성할 때 사용하는 함수입니다.
    원하는 이미지 크기와 배치 사이즈를 정하면 생성된 이미지 리스트를 반환합니다.
    """
    return p_sample_loop(model, shape=(batch_size, channels, image_size, image_size))

## 12. 모델, Optimizer, 결과 저장 폴더 설정

이제까지 정의한 U-Net과 손실 함수를 이용하여 **실제 학습 루프를 돌릴 준비**를 합니다.

- base dimension: `image_size` (28) 을 그대로 사용
- `dim_mults`: `(1, 2, 4)` 로 해서 U-Net 깊이를 적당히 설정
- Optimizer: Adam, learning rate = 1e-3


In [ ]:
# -------------------------------------------------------------------
# 1. 결과 저장 경로 설정
# -------------------------------------------------------------------
# 학습 중간에 생성된 샘플 이미지와 모델 체크포인트를 저장할 폴더를 지정합니다.
results_folder = Path("./results_ddpm")
results_folder.mkdir(exist_ok=True) # 폴더가 없으면 생성하고, 있으면 그대로 사용합니다.

# 400 step마다 현재 모델의 성능을 확인하기 위해 이미지를 생성(Sampling)하고 저장합니다.
save_and_sample_every = 400

# -------------------------------------------------------------------
# 2. U-Net 모델 선언
# -------------------------------------------------------------------
# Diffusion의 핵심 두뇌인 U-Net 신경망을 생성합니다.
model = Unet(
    dim=image_size,           # 기본 특징 맵(Feature map)의 크기 (Fashion-MNIST의 경우 28)
    channels=channels,        # 입력 이미지의 채널 수 (흑백이므로 1)
    dim_mults=(1, 2, 4),      # Downsampling을 진행하며 채널 수를 몇 배씩 늘릴지 결정 (1배 -> 2배 -> 4배)
    resnet_block_groups=4,    # Group Normalization에 사용될 그룹 수 (학습 안정화에 도움)
)

# 모델을 GPU(device)로 이동시켜 연산 속도를 높입니다.
model = model.to(device)

# -------------------------------------------------------------------
# 3. 최적화 도구(Optimizer) 설정
# -------------------------------------------------------------------
# 모델의 가중치를 업데이트할 알고리즘으로 Adam을 사용합니다.
# lr=1e-3 (0.001)은 모델이 한 번에 얼마나 크게 학습할지를 결정하는 학습률입니다.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# -------------------------------------------------------------------
# 4. 모델 규모 확인
# -------------------------------------------------------------------
# 전체 모델의 파라미터(가중치) 개수를 출력하여 모델의 크기를 가늠합니다.
# numel()은 각 텐서의 요소 개수를 반환합니다.
print("Model parameters:", sum(p.numel() for p in model.parameters()))

## 13. 학습 루프 구현

### 13.1 학습 절차

각 minibatch 에 대해 다음을 수행합니다.

1. 이미지 배치 \(x_0\) 를 로드
2. 각 샘플마다 time step \(t \sim \mathcal{U}({0, \dots, T-1})\) 를 샘플링
3. `p_losses` 를 통해 loss를 계산
4. 역전파 및 optimizer step
5. 일정 step마다 `sample` 함수를 호출하여 중간 결과 이미지 저장



In [ ]:
# -------------------------------------------------------------------
# DDPM 메인 학습 루프
# -------------------------------------------------------------------

epochs = 10  # 전체 데이터셋을 몇 번 반복해서 학습할지 설정

for epoch in range(epochs):
    model.train()
    print(f"Epoch {epoch + 1}/{epochs}")

    # DataLoader를 통해 배치 단위로 데이터를 가져옵니다.
    for step, batch in enumerate(dataloader):
        # 1. 변화도(Gradient) 초기화: 이전 루프의 기울기 값을 비웁니다.
        optimizer.zero_grad()

        # 현재 배치의 이미지 개수 확인 및 데이터를 GPU로 전송
        batch_imgs = batch["pixel_values"].to(device)
        b = batch_imgs.shape[0]

        # 2. 타임스텝(t) 샘플링: 배치 내 각 이미지마다 서로 다른 t를 무작위로 선택합니다.
        t = torch.randint(0, timesteps, (b,), device=device, dtype=torch.long)

        # 3. 손실 함수 계산: p_losses 함수 내에서 노이즈 주입(Forward) 및 노이즈 예측이 일어납니다.
        # 여기서는 오차에 강건한 'huber' loss를 사용합니다.
        loss = p_losses(model, batch_imgs, t, loss_type="huber")

        # 100 step마다 현재 학습 손실(Loss)을 출력하여 모니터링합니다.
        if step % 100 == 0:
            print(f"   Step {step:05d} | Loss: {loss.item():.4f}")

        # 4. 역전파(Backpropagation): 손실값으로부터 가중치별 기울기를 계산합니다.
        loss.backward()

        # 5. 가중치 업데이트: 계산된 기울기를 바탕으로 optimizer가 모델의 파라미터를 수정합니다.
        optimizer.step()

        # ---- sampling & save ----
        if step != 0 and step % save_and_sample_every == 0:
            milestone = step // save_and_sample_every

            was_training = model.training
            model.eval()

            # 생성할 이미지 개수(4개)를 배치 사이즈에 맞게 그룹화합니다.
            # (예: 4개를 생성해야 하는데 배치가 128이면 [4]라는 리스트 생성)
            groups = num_to_groups(num=4, divisor=b)

            # 학습 중인 모델을 사용하여 실제 이미지를 생성해봅니다 (Reverse Diffusion).
            # sample() 함수의 결과 중 마지막 단계([-1])인 x_0를 가져옵니다.            
            all_images_list = [
                sample(model, image_size=image_size, batch_size=n, channels=channels)[-1]
                for n in groups
            ]

            # 리스트에 담긴 넘파이 배열들을 하나로 합치고 텐서로 변환합니다.
            all_images = np.concatenate(all_images_list, axis=0)     # (4,1,28,28) numpy
            all_images = torch.from_numpy(all_images).float()        # torch float32 CPU
            # 후처리: 시각화를 위해 [-1, 1] 범위를 다시 [0, 1] 범위로 되돌립니다.
            all_images = ((all_images + 1) * 0.5).clamp(0, 1)        # [0,1]

            # 생성된 이미지를 파일로 저장하여 학습 진행 상황을 눈으로 확인합니다.
            save_path = str(results_folder / f"sample-epoch{epoch+1}-step{step}-milestone{milestone}.png")
            save_image(all_images, save_path, nrow=4)
            print(f"   [Saved] {save_path}")

            if was_training:
                model.train()



## 14. 학습 후 샘플링 및 시각화

학습이 완료된 후, **모델이 학습한 분포에서 샘플을 생성**해 봅니다.

1. `sample` 함수를 이용해 노이즈에서부터 T step reverse diffusion
2. 최종 결과 \(x_0\) 를 가져와 시각화
3. 랜덤 샘플 몇 개를 그레이스케일 이미지로 표시


In [ ]:
# -------------------------------------------------------------------
# 1. 이미지 샘플링 실행
# -------------------------------------------------------------------
# 학습된 U-Net 모델을 사용하여 64장의 이미지를 동시에 생성합니다.
# samples는 리스트 형태이며, 각 원소는 [T-1, T-2, ..., 0] 단계의 이미지들입니다.
samples = sample(
    model,
    image_size=image_size,
    batch_size=64,
    channels=channels,
)

# -------------------------------------------------------------------
# 2. 결과물 선택 및 후처리
# -------------------------------------------------------------------
# samples[-1]은 가장 마지막 단계(t=0)인 최종 완성본입니다. -> (64, 1, 28, 28)
final_samples = samples[-1]

# 넘파이(numpy) 배열 형태인 결과를 연산을 위해 파이토치 텐서로 변환합니다.
final_samples_t = torch.tensor(final_samples)

# 모델의 출력 범위인 [-1, 1]을 시각화 가능한 범위인 [0, 1]로 조정합니다.
final_samples_t = (final_samples_t + 1) * 0.5


# -------------------------------------------------------------------
# 3. 무작위 3장 선택 및 시각화
# -------------------------------------------------------------------
# 0~63 사이의 인덱스 중 중복 없이 3개를 무작위로 뽑습니다.
import random
num_to_show = 3
random_indices = random.sample(range(64), num_to_show)

# 1행 3열의 서브플롯 생성
fig, axes = plt.subplots(1, num_to_show, figsize=(12, 4))

for i, idx in enumerate(random_indices):
    # 이미지 데이터를 (1, 28, 28)에서 (28, 28)로 리셰이프하여 2차원 평면으로 만듭니다.
    axes[i].imshow(
        final_samples_t[idx].reshape(image_size, image_size),
        cmap="gray"  # Fashion-MNIST이므로 흑백 지도를 사용합니다.
    )
    axes[i].set_title(f"Sample Index: {idx}") # 몇 번째 이미지인지 표시
    axes[i].axis("off") # 축 숨기기

plt.tight_layout()
plt.show()

## 15. Denoising 과정 GIF로 만들기 (선택)

마지막으로, 한 샘플에 대해 \(x_T \to x_0\) 로 점점 **노이즈가 제거되는 과정**을 GIF 로 저장해 볼 수 있습니다.

> 이 부분은 강의에서 “모든 step의 중간 결과”를 직관적으로 보여줄 때 매우 유용합니다.


In [ ]:
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from IPython.display import Image as IPyImage

# -------------------------------------------------------------------
# 1. 애니메이션 설정 및 프레임 생성
# -------------------------------------------------------------------
# 64장의 생성 결과 중 애니메이션으로 보고 싶은 이미지의 인덱스를 선택합니다.
random_index = 0
fig = plt.figure()
ims = []

# 전체 타임스텝(T) 동안의 변화 과정을 하나씩 이미지(프레임)로 만듭니다.
for i in range(timesteps):
    # i번째 타임스텝의 결과물 중 random_index번째 이미지를 가져와 2차원으로 변환
    img = samples[i][random_index].reshape(image_size, image_size)

    # [0, 1] 범위로 정규화하여 출력 (samples가 이미 전처리되어 있다면 그대로 사용)
    # plt.imshow를 실행하고 'animated=True' 설정을 통해 애니메이션용 객체로 저장합니다.
    im = plt.imshow(img, cmap="gray", animated=True)

    # 각 프레임을 리스트에 차곡차곡 쌓습니다.
    ims.append([im])

# -------------------------------------------------------------------
# 2. ArtistAnimation 생성
# -------------------------------------------------------------------
# interval: 프레임 간의 간격 (ms 단위, 50ms = 0.05초)
# blit: 성능 최적화 옵션 (True 설정 시 변경된 부분만 다시 그림)
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

# -------------------------------------------------------------------
# 3. GIF 파일 저장 및 출력
# -------------------------------------------------------------------
# 결과를 저장할 경로 지정
gif_path = results_folder / "diffusion_process.gif"

# PillowWriter를 사용하여 GIF로 저장합니다. fps는 초당 프레임 수를 의미합니다.
ani.save(str(gif_path), writer=PillowWriter(fps=20))

# 저장이 끝난 후 불필요해진 figure 객체를 닫아 메모리를 관리합니다.
plt.close(fig)

# 생성된 GIF 파일을 주피터 노트북 화면에 즉시 표시합니다.
display(IPyImage(filename=str(gif_path)))